# 2. Feature Engineering

# Rationale

In this part of the notebook, I wanted to add in more features to enable the models to be more robust in the HDB price prediction. Therefore, I am thinking to add in the following features:

- year transacted (this can be pulled from the month column which includes the month and year of transaction date)
- age (calculated by taking current year 2021 minus lease commencement date)
- inflation adjusted resale price (using Singapore's historical inflation data to adjust the prices accordingly to 2021's value)
- price per sqf (taking nominal prices divided by HDB area)
- floor area in sqf (the housing market in Singapore more commonly uses sqf instead of sqm as a comparison metric)
- distance to nearby amenities (being a real estate class, location is important and this may add more meaningful context to pricing trend)
- storey (the initial data provided a floor range for each transaction. We will convert it into numbers by taking the average of the range of numbers

# Imports

In [1]:
import pandas as pd
import requests
import numpy as np
import math

In [7]:
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 200)

# Read in HDB data with geo coordinates

In [37]:
df = pd.read_csv('../datasets/hdb_2015_2016_long_lat.csv')
df2 = pd.read_csv('../datasets/hdb_2012mar_2014dec_long_lat.csv')
df3 = pd.read_csv('../datasets/hdb_2017_recent_long_lat.csv')
df4 = pd.read_csv('../datasets/hdb_2011_oct_to_2012_feb_long_lat.csv')

In [4]:
print(df.shape, df2.shape, df3.shape, df4.shape)

(37153, 13) (52203, 12) (109322, 13) (8120, 12)


In [5]:
print(df.columns)
print(df2.columns)
print(df3.columns)
print(df4.columns)

Index(['month', 'town', 'flat_type', 'block', 'street_name', 'storey_range',
       'floor_area_sqm', 'flat_model', 'lease_commence_date',
       'remaining_lease', 'resale_price', 'longitude', 'latitude'],
      dtype='object')
Index(['month', 'town', 'flat_type', 'block', 'street_name', 'storey_range',
       'floor_area_sqm', 'flat_model', 'lease_commence_date', 'resale_price',
       'longitude', 'latitude'],
      dtype='object')
Index(['month', 'town', 'flat_type', 'block', 'street_name', 'storey_range',
       'floor_area_sqm', 'flat_model', 'lease_commence_date',
       'remaining_lease', 'resale_price', 'longitude', 'latitude'],
      dtype='object')
Index(['month', 'town', 'flat_type', 'block', 'street_name', 'storey_range',
       'floor_area_sqm', 'flat_model', 'lease_commence_date', 'resale_price',
       'longitude', 'latitude'],
      dtype='object')


df2 is 1 column short from the other dfs which is 'remaining_lease' column. Since we have 'lease_commence_date' column, we can drop the 'remaining_lease' column since they are correlated to each other.

# Data Cleaning

We will drop 'remaining_lease' column since we have 'lease_commence_date' column in the dataframes.

In [6]:
df.drop(columns='remaining_lease', inplace=True)
df3.drop(columns='remaining_lease', inplace=True)

In [7]:
print(df.shape, df2.shape, df3.shape, df4.shape)

(37153, 12) (52203, 12) (109322, 12) (8120, 12)


We note that in the 'flat_model' column that there were also HDB terrace house transactions. For the purpose of this project, we are only interested in HDB apartment unit types and therefore will be dropping these rows.

In [38]:
df = df[~df['flat_model'].str.contains('Terrace')]
df2 = df2[~df2['flat_model'].str.contains('Terrace')]
df3 = df3[~df3['flat_model'].str.contains('Terrace')]
df4 = df4[~df4['flat_model'].str.contains('Terrace')]

In [39]:
print(df.shape, df2.shape, df3.shape, df4.shape)

(37129, 13) (52170, 12) (109251, 13) (8111, 12)


In [40]:
# after removing rows containing 'Terrace' flat models, we would need to reset the index order
df.index = np.arange(len(df))
df2.index = np.arange(len(df2))
df3.index = np.arange(len(df3))
df4.index = np.arange(len(df4))

# Feature Engineering

In order to make our models predict more meaningful, we will create new features on the datasets to make it more robust and give more context to each HDB transaction. In particular, prices of housing (as a form of real estate) is typically driven by location and we will set out to create distance based features to calculate the HDB's proximity to nearby amenities. We will aim to create the following features below:

- Create year transacted column
- Create age of unit
- Modifying resale price to 2021's prices to adjust for inflation 
- Create price per sqf 
- Create proximity to nearest MRT station column
- Create proximity to nearest primary and secondary schools column 
- Create proximity to nearest mall column
- Create proximity to CBD column 
- Modifying storey range column into ordinal variables: higher floor should have a higher value

## Create year transacted column function

In [41]:
def get_year(df):
    '''get the year of transaction for HDB'''
    df['month'] = pd.to_datetime(df['month'])
    df['year_transacted'] = df['month'].dt.year
    return df

In [1]:
#get_year(df)

![](images/get_year(df).jpg)

In [2]:
#get_year(df2)

![](images/get_year(df2).jpg)

In [3]:
#get_year(df3)

![](images/get_year(df3).jpg)

In [4]:
#get_year(df4)

![](images/get_year(df4).jpg)

## Create age function

In [14]:
def age(df):
    '''calculate age of the HDB'''
    df['age'] = 2021 - df['lease_commence_date']
    return df

In [5]:
#age(df)

![](images/age(df).jpg)

In [6]:
#age(df2)

![](images/age(df2).jpg)

In [7]:
#age(df3)

![](images/age(df3).jpg)

In [8]:
#age(df4)

![](images/age(df4).jpg)

## Inflation-adjusted resale price function

In [11]:
inflation = pd.read_csv('../datasets/inflation/consumer-price-index-2019-as-base-year-annual.csv')

In [12]:
inflation = inflation[inflation['level_1'] == 'All Items']

In [13]:
inflation = inflation[inflation['year'] > 2000]

In [14]:
inflation.drop(columns='level_1', inplace=True)

In [15]:
inflation.index = np.arange(len(inflation))

In [16]:
inflation['value'] = inflation['value'].astype(float)

In [17]:
# 1.5% inflation assumed for 2021 based on CNA article: 
# https://www.channelnewsasia.com/singapore/core-inflation-rises-singapore-august-2021-mas-mti-2196586

inflation = inflation.append({'year': 2021,
                  'value': 99.818 * (1+0.015)}, ignore_index=True)

In [18]:
# calculation multiplier rate using 2021 as base year
inflation['multiplier'] = 101.31527 / inflation['value']

In [19]:
inflation['year'] = inflation['year'].astype(int)

In [20]:
inflation

,year,value,multiplier
0,2001,75.19000,1.347457
1,2002,74.89600,1.352746
2,2003,75.26100,1.346186
3,2004,76.51800,1.324071
4,2005,76.87700,1.317888
5,2006,77.61700,1.305323
6,2007,79.25100,1.278410
7,2008,84.50400,1.198941
8,2009,85.00800,1.191832
9,2010,87.40800,1.159108


In [23]:
inflation.set_index('year', inplace=True)
inflation

,value,multiplier
year,,
2001,75.19000,1.347457
2002,74.89600,1.352746
2003,75.26100,1.346186
2004,76.51800,1.324071
2005,76.87700,1.317888
2006,77.61700,1.305323
2007,79.25100,1.278410
2008,84.50400,1.198941
2009,85.00800,1.191832


In [46]:
inflation.to_csv('../datasets/inflation/inflation_cleaned.csv', index=False)

In [44]:
# inflation-adjusted resale price function
def price_adjust(row):
    row['year_transacted'] = row['resale_price'] * inflation.loc[row['year_transacted'], 'multiplier']
    return row

In [9]:
#df = df.apply(price_adjust, axis=1)
#df

![](images/price_adjust(df).jpg)

In [10]:
#df2 = df2.apply(price_adjust, axis=1)
#df2

![](images/price_adjust(df2).jpg)

In [11]:
#df3 = df3.apply(price_adjust, axis=1)
#df3

![](images/price_adjust(df3).jpg)

In [12]:
#df4 = df4.apply(price_adjust, axis=1)
#df4

![](images/price_adjust(df4).jpg)

Note that the cell above takes a while to run as there are many rows in the HDB dfs.

## Price per sqf function

Note that the price per sqf is based on the nominal value of resale price and not the inflation adjusted resale price.

In [5]:
def psf(df):
    df['psf'] = df['resale_price'] / df['floor_area_sqm'] / 10.7639
    return df

In [13]:
#psf(df)

![](images/psf(df).jpg)

In [14]:
#psf(df2)

![](images/psf(df2).jpg)

In [15]:
#psf(df3)

![](images/psf(df3).jpg)

In [16]:
#psf(df4)

![](images/psf(df4).jpg)

## floor_area_sqf function

In [31]:
def area_sqf(row):
    '''creates new column: floor_area_sqf from existing floor_area_sqm'''
    row['floor_area_sqf'] = row['floor_area_sqm'] * 10.7639
    return row

In [18]:
#df = df.apply(area_sqf, axis=1)
#df

![](images/area(df).jpg)

In [31]:
df2 = df2.apply(area_sqf, axis=1)
df2

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,longitude,latitude,year_transacted,age,resale_price_inflate,psf,floor_area_sqf
0,2012-03-01,ANG MO KIO,2 ROOM,172,ANG MO KIO AVE 4,06 TO 10,45.0,Improved,1986,250000.0,103.836463,1.374694,2012,35,263279.637233,543.544496,484.3755
1,2012-03-01,ANG MO KIO,2 ROOM,510,ANG MO KIO AVE 8,01 TO 05,44.0,Improved,1980,265000.0,103.849073,1.373401,2012,41,279076.415467,589.251647,473.6116
2,2012-03-01,ANG MO KIO,3 ROOM,610,ANG MO KIO AVE 4,06 TO 10,68.0,New Generation,1980,315000.0,103.839157,1.379395,2012,41,331732.342914,453.220190,731.9452
3,2012-03-01,ANG MO KIO,3 ROOM,474,ANG MO KIO AVE 10,01 TO 05,67.0,New Generation,1984,320000.0,103.858015,1.362758,2012,37,336997.935658,467.286015,721.1813
4,2012-03-01,ANG MO KIO,3 ROOM,604,ANG MO KIO AVE 5,06 TO 10,67.0,New Generation,1980,321000.0,103.835977,1.379867,2012,41,338051.054207,468.746284,721.1813
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52165,2014-12-01,YISHUN,5 ROOM,816,YISHUN ST 81,10 TO 12,122.0,Improved,1988,580000.0,103.833368,1.411771,2014,33,590682.393977,449.805272,1313.1958
52166,2014-12-01,YISHUN,EXECUTIVE,325,YISHUN CTRL,10 TO 12,146.0,Maisonette,1988,540000.0,103.842146,1.429239,2014,33,549945.677151,349.942977,1571.5294
52167,2014-12-01,YISHUN,EXECUTIVE,618,YISHUN RING RD,07 TO 09,164.0,Apartment,1992,738000.0,103.835703,1.418735,2014,29,751592.425440,425.763956,1765.2796
52168,2014-12-01,YISHUN,EXECUTIVE,277,YISHUN ST 22,07 TO 09,152.0,Maisonette,1985,592000.0,103.836995,1.437918,2014,36,602903.409025,368.497459,1636.1128


![](images/area(df2).jpg)

In [55]:
df3 = df3.apply(area_sqf, axis=1)
df3

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,longitude,latitude,year_transacted,age,resale_price_inflate,psf,floor_area_sqf
0,2017-01-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,232000.0,103.853880,1.362005,2017,42,237416.090663,501.288589,473.6116
1,2017-01-01,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67.0,New Generation,1978,250000.0,103.837975,1.370943,2017,43,255836.304594,354.746171,721.1813
2,2017-01-01,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,262000.0,103.835368,1.380709,2017,41,268116.447214,371.773987,721.1813
3,2017-01-01,ANG MO KIO,3 ROOM,465,ANG MO KIO AVE 10,04 TO 06,68.0,New Generation,1980,265000.0,103.857201,1.366201,2017,41,271186.482869,370.501074,731.9452
4,2017-01-01,ANG MO KIO,3 ROOM,601,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,265000.0,103.835132,1.381041,2017,41,271186.482869,376.030941,721.1813
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109246,2021-10-01,YISHUN,4 ROOM,458,YISHUN AVE 11,10 TO 12,93.0,Model A,2013,468888.0,103.846208,1.420667,2021,8,468888.000000,468.399600,1001.0427
109247,2021-10-01,YISHUN,4 ROOM,621,YISHUN RING RD,07 TO 09,107.0,Model A,1988,495000.0,103.835187,1.418667,2021,33,495000.000000,429.785508,1151.7373
109248,2021-10-01,YISHUN,4 ROOM,512B,YISHUN ST 51,04 TO 06,93.0,Model A,2017,475000.0,103.842112,1.414707,2021,4,475000.000000,474.505233,1001.0427
109249,2021-10-01,YISHUN,5 ROOM,342A,YISHUN RING RD,04 TO 06,117.0,Premium Apartment,2016,611888.0,103.844465,1.430597,2021,5,611888.000000,485.865900,1259.3763


![](images/area(df3).jpg)

In [32]:
df4 = df4.apply(area_sqf, axis=1)
df4

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,longitude,latitude,year_transacted,age,resale_price_inflate,psf,floor_area_sqf
0,2011-10-01,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,01 TO 03,60.0,Improved,1986,283000.0,103.837619,1.375097,2011,35,311671.519213,482.587661,645.8340
1,2011-10-01,ANG MO KIO,3 ROOM,331,ANG MO KIO AVE 1,10 TO 12,68.0,New Generation,1981,343000.0,103.850766,1.362111,2011,40,377750.286537,516.090940,731.9452
2,2011-10-01,ANG MO KIO,3 ROOM,441,ANG MO KIO AVE 10,04 TO 06,67.0,New Generation,1979,333000.0,103.854168,1.366050,2011,42,366737.158650,508.522834,721.1813
3,2011-10-01,ANG MO KIO,3 ROOM,442,ANG MO KIO AVE 10,10 TO 12,82.0,New Generation,1979,399000.0,103.855232,1.365653,2011,42,439423.802707,497.851788,882.6398
4,2011-10-01,ANG MO KIO,3 ROOM,435,ANG MO KIO AVE 10,04 TO 06,82.0,New Generation,1979,383000.0,103.854214,1.367704,2011,42,421802.798087,477.887807,882.6398
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8106,2012-02-01,YISHUN,5 ROOM,212,YISHUN ST 21,10 TO 12,121.0,Improved,1985,476888.0,103.837438,1.431783,2012,36,502219.598563,385.601427,1302.4319
8107,2012-02-01,YISHUN,5 ROOM,758,YISHUN ST 72,01 TO 03,122.0,Improved,1986,490000.0,103.834266,1.426030,2012,35,516028.088977,392.955939,1313.1958
8108,2012-02-01,YISHUN,5 ROOM,873,YISHUN ST 81,01 TO 03,122.0,Improved,1988,488000.0,103.836950,1.414695,2012,33,513921.851879,391.352037,1313.1958
8109,2012-02-01,YISHUN,EXECUTIVE,664,YISHUN AVE 4,07 TO 09,181.0,Apartment,1992,705000.0,103.841075,1.420200,2012,29,742448.576997,381.081749,1948.2659


![](images/area(df4).jpg)

## Distance function

In [25]:
def get_distance_km_amenity(hdb_df, amenity_df, amenity_name):
    
    '''
    compare with amenity df (e.g. mrt, mall, schools) and hdb df, calculate the shortest distance to nearest amenity
    return nearest amenity name and amenity distance
    '''
    hdb_df[f'nearest_{amenity_name}_dist'] = ''
    hdb_df[f'nearest_{amenity_name}'] = ''
    
    row = 0
    
    # looping through hdb df's longitude and latitude columns to pull geo coordinates
    for hdb_long, hdb_lat in zip(hdb_df['longitude'], hdb_df['latitude']):
        lat1 = math.radians(hdb_lat)
        lon1 = math.radians(hdb_long)

        
        nearest_amenity = ''
        nearest_amenity_dist = math.inf
        temp_dist_list = []
        temp_amenity_list = []

        for long, lat in zip(amenity_df['longitude'], amenity_df['latitude']): 

            lat2 = math.radians(lat)
            lon2 = math.radians(long)
            nearest_amenity = np.array(amenity_df.loc[(amenity_df['longitude'] == long) & (amenity_df['latitude'] == lat), amenity_name])[0]

            dlon = lon2 - lon1
            dlat = lat2 - lat1

            # Using Haversine formula
            a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
            c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
            dist = 6373 * c

            temp_dist_list.append(dist)
            temp_amenity_list.append(nearest_amenity)

        # create a dictionary with key = amenity name, value = dist between amenity and hdb
        result = dict(zip(temp_amenity_list, temp_dist_list))
        
        # fill nearest dist in hdb df with the lowest dictionary value
        hdb_df[f'nearest_{amenity_name}_dist'][row] = min(result.values())

        # fill nearest amenity name in hdb df with the corresponding dictionary key 
        for key, value in result.items():
            if value == min(result.values()):
                print(f'HDB row {row} checking against amenity df..')
                hdb_df[f'nearest_{amenity_name}'][row] = key

        row += 1   
        
    return hdb_df
    

### Add nearest mrt to HDB df

In [27]:
mrt = pd.read_csv('../datasets/amenities/mrt_cleaned.csv')

In [19]:
#get_distance_km_amenity(df, mrt, 'mrt_station')

![](images/get_dist.jpg)

In [20]:
#get_distance_km_amenity(df2, mrt, 'mrt_station')

![](images/get_dist.jpg)

In [21]:
#get_distance_km_amenity(df3, mrt, 'mrt_station')

![](images/get_dist.jpg)

In [35]:
get_distance_km_amenity(df4, mrt, 'mrt_station')

C:\Users\caspe\anaconda3\envs\capstone\lib\site-packages\ipykernel_launcher.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\caspe\anaconda3\envs\capstone\lib\site-packages\ipykernel_launcher.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


HDB row 0 checking against amenity df..
HDB row 1 checking against amenity df..
HDB row 2 checking against amenity df..
HDB row 3 checking against amenity df..
HDB row 4 checking against amenity df..
HDB row 5 checking against amenity df..
HDB row 6 checking against amenity df..
HDB row 7 checking against amenity df..
HDB row 8 checking against amenity df..
HDB row 9 checking against amenity df..
HDB row 10 checking against amenity df..
HDB row 11 checking against amenity df..
HDB row 12 checking against amenity df..
HDB row 13 checking against amenity df..
HDB row 14 checking against amenity df..
HDB row 15 checking against amenity df..
HDB row 16 checking against amenity df..
HDB row 17 checking against amenity df..
HDB row 18 checking against amenity df..
HDB row 19 checking against amenity df..
HDB row 20 checking against amenity df..
HDB row 21 checking against amenity df..
HDB row 22 checking against amenity df..
HDB row 23 checking against amenity df..
HDB row 24 checking agains

HDB row 198 checking against amenity df..
HDB row 199 checking against amenity df..
HDB row 200 checking against amenity df..
HDB row 201 checking against amenity df..
HDB row 202 checking against amenity df..
HDB row 203 checking against amenity df..
HDB row 204 checking against amenity df..
HDB row 205 checking against amenity df..
HDB row 206 checking against amenity df..
HDB row 207 checking against amenity df..
HDB row 208 checking against amenity df..
HDB row 209 checking against amenity df..
HDB row 210 checking against amenity df..
HDB row 211 checking against amenity df..
HDB row 212 checking against amenity df..
HDB row 213 checking against amenity df..
HDB row 214 checking against amenity df..
HDB row 215 checking against amenity df..
HDB row 216 checking against amenity df..
HDB row 217 checking against amenity df..
HDB row 218 checking against amenity df..
HDB row 219 checking against amenity df..
HDB row 220 checking against amenity df..
HDB row 221 checking against ameni

HDB row 396 checking against amenity df..
HDB row 397 checking against amenity df..
HDB row 398 checking against amenity df..
HDB row 399 checking against amenity df..
HDB row 400 checking against amenity df..
HDB row 401 checking against amenity df..
HDB row 402 checking against amenity df..
HDB row 403 checking against amenity df..
HDB row 404 checking against amenity df..
HDB row 405 checking against amenity df..
HDB row 406 checking against amenity df..
HDB row 407 checking against amenity df..
HDB row 408 checking against amenity df..
HDB row 409 checking against amenity df..
HDB row 410 checking against amenity df..
HDB row 411 checking against amenity df..
HDB row 412 checking against amenity df..
HDB row 413 checking against amenity df..
HDB row 414 checking against amenity df..
HDB row 415 checking against amenity df..
HDB row 416 checking against amenity df..
HDB row 417 checking against amenity df..
HDB row 418 checking against amenity df..
HDB row 419 checking against ameni

HDB row 593 checking against amenity df..
HDB row 594 checking against amenity df..
HDB row 595 checking against amenity df..
HDB row 596 checking against amenity df..
HDB row 597 checking against amenity df..
HDB row 598 checking against amenity df..
HDB row 599 checking against amenity df..
HDB row 600 checking against amenity df..
HDB row 601 checking against amenity df..
HDB row 602 checking against amenity df..
HDB row 603 checking against amenity df..
HDB row 604 checking against amenity df..
HDB row 605 checking against amenity df..
HDB row 606 checking against amenity df..
HDB row 607 checking against amenity df..
HDB row 608 checking against amenity df..
HDB row 609 checking against amenity df..
HDB row 610 checking against amenity df..
HDB row 611 checking against amenity df..
HDB row 612 checking against amenity df..
HDB row 613 checking against amenity df..
HDB row 614 checking against amenity df..
HDB row 615 checking against amenity df..
HDB row 616 checking against ameni

...

HDB row 7365 checking against amenity df..
HDB row 7366 checking against amenity df..
HDB row 7367 checking against amenity df..
HDB row 7368 checking against amenity df..
HDB row 7369 checking against amenity df..
HDB row 7370 checking against amenity df..
HDB row 7371 checking against amenity df..
HDB row 7372 checking against amenity df..
HDB row 7373 checking against amenity df..
HDB row 7374 checking against amenity df..
HDB row 7375 checking against amenity df..
HDB row 7376 checking against amenity df..
HDB row 7377 checking against amenity df..
HDB row 7378 checking against amenity df..
HDB row 7379 checking against amenity df..
HDB row 7380 checking against amenity df..
HDB row 7381 checking against amenity df..
HDB row 7382 checking against amenity df..
HDB row 7383 checking against amenity df..
HDB row 7384 checking against amenity df..
HDB row 7385 checking against amenity df..
HDB row 7386 checking against amenity df..
HDB row 7387 checking against amenity df..
HDB row 738

HDB row 7559 checking against amenity df..
HDB row 7560 checking against amenity df..
HDB row 7561 checking against amenity df..
HDB row 7562 checking against amenity df..
HDB row 7563 checking against amenity df..
HDB row 7564 checking against amenity df..
HDB row 7565 checking against amenity df..
HDB row 7566 checking against amenity df..
HDB row 7567 checking against amenity df..
HDB row 7568 checking against amenity df..
HDB row 7569 checking against amenity df..
HDB row 7570 checking against amenity df..
HDB row 7571 checking against amenity df..
HDB row 7572 checking against amenity df..
HDB row 7573 checking against amenity df..
HDB row 7574 checking against amenity df..
HDB row 7575 checking against amenity df..
HDB row 7576 checking against amenity df..
HDB row 7577 checking against amenity df..
HDB row 7578 checking against amenity df..
HDB row 7579 checking against amenity df..
HDB row 7580 checking against amenity df..
HDB row 7581 checking against amenity df..
HDB row 758

HDB row 7752 checking against amenity df..
HDB row 7753 checking against amenity df..
HDB row 7754 checking against amenity df..
HDB row 7755 checking against amenity df..
HDB row 7756 checking against amenity df..
HDB row 7757 checking against amenity df..
HDB row 7758 checking against amenity df..
HDB row 7759 checking against amenity df..
HDB row 7760 checking against amenity df..
HDB row 7761 checking against amenity df..
HDB row 7762 checking against amenity df..
HDB row 7763 checking against amenity df..
HDB row 7764 checking against amenity df..
HDB row 7765 checking against amenity df..
HDB row 7766 checking against amenity df..
HDB row 7767 checking against amenity df..
HDB row 7768 checking against amenity df..
HDB row 7769 checking against amenity df..
HDB row 7770 checking against amenity df..
HDB row 7771 checking against amenity df..
HDB row 7772 checking against amenity df..
HDB row 7773 checking against amenity df..
HDB row 7774 checking against amenity df..
HDB row 777

HDB row 7945 checking against amenity df..
HDB row 7946 checking against amenity df..
HDB row 7947 checking against amenity df..
HDB row 7948 checking against amenity df..
HDB row 7949 checking against amenity df..
HDB row 7950 checking against amenity df..
HDB row 7951 checking against amenity df..
HDB row 7952 checking against amenity df..
HDB row 7953 checking against amenity df..
HDB row 7954 checking against amenity df..
HDB row 7955 checking against amenity df..
HDB row 7956 checking against amenity df..
HDB row 7957 checking against amenity df..
HDB row 7958 checking against amenity df..
HDB row 7959 checking against amenity df..
HDB row 7960 checking against amenity df..
HDB row 7961 checking against amenity df..
HDB row 7962 checking against amenity df..
HDB row 7963 checking against amenity df..
HDB row 7964 checking against amenity df..
HDB row 7965 checking against amenity df..
HDB row 7966 checking against amenity df..
HDB row 7967 checking against amenity df..
HDB row 796

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,longitude,latitude,year_transacted,age,resale_price_inflate,psf,floor_area_sqf,nearest_mrt_station_dist,nearest_mrt_station
0,2011-10-01,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,01 TO 03,60.0,Improved,1986,283000.0,103.837619,1.375097,2011,35,311671.519213,482.587661,645.8340,0.420801,MAYFLOWER
1,2011-10-01,ANG MO KIO,3 ROOM,331,ANG MO KIO AVE 1,10 TO 12,68.0,New Generation,1981,343000.0,103.850766,1.362111,2011,40,377750.286537,516.090940,731.9452,0.880294,ANG MO KIO
2,2011-10-01,ANG MO KIO,3 ROOM,441,ANG MO KIO AVE 10,04 TO 06,67.0,New Generation,1979,333000.0,103.854168,1.366050,2011,42,366737.158650,508.522834,721.1813,0.670305,ANG MO KIO
3,2011-10-01,ANG MO KIO,3 ROOM,442,ANG MO KIO AVE 10,10 TO 12,82.0,New Generation,1979,399000.0,103.855232,1.365653,2011,42,439423.802707,497.851788,882.6398,0.790373,ANG MO KIO
4,2011-10-01,ANG MO KIO,3 ROOM,435,ANG MO KIO AVE 10,04 TO 06,82.0,New Generation,1979,383000.0,103.854214,1.367704,2011,42,421802.798087,477.887807,882.6398,0.574072,ANG MO KIO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8106,2012-02-01,YISHUN,5 ROOM,212,YISHUN ST 21,10 TO 12,121.0,Improved,1985,476888.0,103.837438,1.431783,2012,36,502219.598563,385.601427,1302.4319,0.375451,YISHUN
8107,2012-02-01,YISHUN,5 ROOM,758,YISHUN ST 72,01 TO 03,122.0,Improved,1986,490000.0,103.834266,1.426030,2012,35,516028.088977,392.955939,1313.1958,0.388479,YISHUN
8108,2012-02-01,YISHUN,5 ROOM,873,YISHUN ST 81,01 TO 03,122.0,Improved,1988,488000.0,103.836950,1.414695,2012,33,513921.851879,391.352037,1313.1958,0.533236,KHATIB
8109,2012-02-01,YISHUN,EXECUTIVE,664,YISHUN AVE 4,07 TO 09,181.0,Apartment,1992,705000.0,103.841075,1.420200,2012,29,742448.576997,381.081749,1948.2659,0.95306,KHATIB


### Add nearest primary school to HDB df

In [29]:
pri_sch = pd.read_csv('../datasets/amenities/pri_sch_cleaned.csv')

In [37]:
pri_sch

,primary_school,longitude,latitude
0,ZHONGHUA PRIMARY SCHOOL,103.869762,1.360203
1,ZHENGHUA PRIMARY SCHOOL,103.769314,1.379549
2,ZHANGDE PRIMARY SCHOOL,103.825952,1.284212
3,YUMIN PRIMARY SCHOOL,103.950462,1.351292
4,YUHUA PRIMARY SCHOOL,103.741106,1.342802
5,YU NENG PRIMARY SCHOOL,103.932145,1.333889
6,JIEMIN PRIMARY SCHOOL,103.830425,1.427688
7,XISHAN PRIMARY SCHOOL,103.837755,1.433472
8,AHMAD IBRAHIM PRIMARY SCHOOL,103.832942,1.433153
9,YISHUN PRIMARY SCHOOL,103.834087,1.433386


In [22]:
#get_distance_km_amenity(df, pri_sch, 'primary_school')

![](images/get_dist.jpg)

In [23]:
#get_distance_km_amenity(df2, pri_sch, 'primary_school')

![](images/get_dist.jpg)

In [24]:
#get_distance_km_amenity(df3, pri_sch, 'primary_school')

![](images/get_dist.jpg)

In [38]:
get_distance_km_amenity(df4, pri_sch, 'primary_school')

C:\Users\caspe\anaconda3\envs\capstone\lib\site-packages\ipykernel_launcher.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\caspe\anaconda3\envs\capstone\lib\site-packages\ipykernel_launcher.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


HDB row 0 checking against amenity df..
HDB row 1 checking against amenity df..
HDB row 2 checking against amenity df..
HDB row 3 checking against amenity df..
HDB row 4 checking against amenity df..
HDB row 5 checking against amenity df..
HDB row 6 checking against amenity df..
HDB row 7 checking against amenity df..
HDB row 8 checking against amenity df..
HDB row 9 checking against amenity df..
HDB row 10 checking against amenity df..
HDB row 11 checking against amenity df..
HDB row 12 checking against amenity df..
HDB row 13 checking against amenity df..
HDB row 14 checking against amenity df..
HDB row 15 checking against amenity df..
HDB row 16 checking against amenity df..
HDB row 17 checking against amenity df..
HDB row 18 checking against amenity df..
HDB row 19 checking against amenity df..
HDB row 20 checking against amenity df..
HDB row 21 checking against amenity df..
HDB row 22 checking against amenity df..
HDB row 23 checking against amenity df..
HDB row 24 checking agains

HDB row 198 checking against amenity df..
HDB row 199 checking against amenity df..
HDB row 200 checking against amenity df..
HDB row 201 checking against amenity df..
HDB row 202 checking against amenity df..
HDB row 203 checking against amenity df..
HDB row 204 checking against amenity df..
HDB row 205 checking against amenity df..
HDB row 206 checking against amenity df..
HDB row 207 checking against amenity df..
HDB row 208 checking against amenity df..
HDB row 209 checking against amenity df..
HDB row 210 checking against amenity df..
HDB row 211 checking against amenity df..
HDB row 212 checking against amenity df..
HDB row 213 checking against amenity df..
HDB row 214 checking against amenity df..
HDB row 215 checking against amenity df..
HDB row 216 checking against amenity df..
HDB row 217 checking against amenity df..
HDB row 218 checking against amenity df..
HDB row 219 checking against amenity df..
HDB row 220 checking against amenity df..
HDB row 221 checking against ameni

HDB row 397 checking against amenity df..
HDB row 398 checking against amenity df..
HDB row 399 checking against amenity df..
HDB row 400 checking against amenity df..
HDB row 401 checking against amenity df..
HDB row 402 checking against amenity df..
HDB row 403 checking against amenity df..
HDB row 404 checking against amenity df..
HDB row 405 checking against amenity df..
HDB row 406 checking against amenity df..
HDB row 407 checking against amenity df..
HDB row 408 checking against amenity df..
HDB row 409 checking against amenity df..
HDB row 410 checking against amenity df..
HDB row 411 checking against amenity df..
HDB row 412 checking against amenity df..
HDB row 413 checking against amenity df..
HDB row 414 checking against amenity df..
HDB row 415 checking against amenity df..
HDB row 416 checking against amenity df..
HDB row 417 checking against amenity df..
HDB row 418 checking against amenity df..
HDB row 419 checking against amenity df..
HDB row 420 checking against ameni

HDB row 595 checking against amenity df..
HDB row 596 checking against amenity df..
HDB row 597 checking against amenity df..
HDB row 598 checking against amenity df..
HDB row 599 checking against amenity df..
HDB row 600 checking against amenity df..
HDB row 601 checking against amenity df..
HDB row 602 checking against amenity df..
HDB row 603 checking against amenity df..
HDB row 604 checking against amenity df..
HDB row 605 checking against amenity df..
HDB row 606 checking against amenity df..
HDB row 607 checking against amenity df..
HDB row 608 checking against amenity df..
HDB row 609 checking against amenity df..
HDB row 610 checking against amenity df..
HDB row 611 checking against amenity df..
HDB row 612 checking against amenity df..
HDB row 613 checking against amenity df..
HDB row 614 checking against amenity df..
HDB row 615 checking against amenity df..
HDB row 616 checking against amenity df..
HDB row 617 checking against amenity df..
HDB row 618 checking against ameni

...

HDB row 7361 checking against amenity df..
HDB row 7362 checking against amenity df..
HDB row 7363 checking against amenity df..
HDB row 7364 checking against amenity df..
HDB row 7365 checking against amenity df..
HDB row 7366 checking against amenity df..
HDB row 7367 checking against amenity df..
HDB row 7368 checking against amenity df..
HDB row 7369 checking against amenity df..
HDB row 7370 checking against amenity df..
HDB row 7371 checking against amenity df..
HDB row 7372 checking against amenity df..
HDB row 7373 checking against amenity df..
HDB row 7374 checking against amenity df..
HDB row 7375 checking against amenity df..
HDB row 7376 checking against amenity df..
HDB row 7377 checking against amenity df..
HDB row 7378 checking against amenity df..
HDB row 7379 checking against amenity df..
HDB row 7380 checking against amenity df..
HDB row 7381 checking against amenity df..
HDB row 7382 checking against amenity df..
HDB row 7383 checking against amenity df..
HDB row 738

HDB row 7552 checking against amenity df..
HDB row 7553 checking against amenity df..
HDB row 7554 checking against amenity df..
HDB row 7555 checking against amenity df..
HDB row 7556 checking against amenity df..
HDB row 7557 checking against amenity df..
HDB row 7558 checking against amenity df..
HDB row 7559 checking against amenity df..
HDB row 7560 checking against amenity df..
HDB row 7561 checking against amenity df..
HDB row 7562 checking against amenity df..
HDB row 7563 checking against amenity df..
HDB row 7564 checking against amenity df..
HDB row 7565 checking against amenity df..
HDB row 7566 checking against amenity df..
HDB row 7567 checking against amenity df..
HDB row 7568 checking against amenity df..
HDB row 7569 checking against amenity df..
HDB row 7570 checking against amenity df..
HDB row 7571 checking against amenity df..
HDB row 7572 checking against amenity df..
HDB row 7573 checking against amenity df..
HDB row 7574 checking against amenity df..
HDB row 757

HDB row 7744 checking against amenity df..
HDB row 7745 checking against amenity df..
HDB row 7746 checking against amenity df..
HDB row 7747 checking against amenity df..
HDB row 7748 checking against amenity df..
HDB row 7749 checking against amenity df..
HDB row 7750 checking against amenity df..
HDB row 7751 checking against amenity df..
HDB row 7752 checking against amenity df..
HDB row 7753 checking against amenity df..
HDB row 7754 checking against amenity df..
HDB row 7755 checking against amenity df..
HDB row 7756 checking against amenity df..
HDB row 7757 checking against amenity df..
HDB row 7758 checking against amenity df..
HDB row 7759 checking against amenity df..
HDB row 7760 checking against amenity df..
HDB row 7761 checking against amenity df..
HDB row 7762 checking against amenity df..
HDB row 7763 checking against amenity df..
HDB row 7764 checking against amenity df..
HDB row 7765 checking against amenity df..
HDB row 7766 checking against amenity df..
HDB row 776

HDB row 7937 checking against amenity df..
HDB row 7938 checking against amenity df..
HDB row 7939 checking against amenity df..
HDB row 7940 checking against amenity df..
HDB row 7941 checking against amenity df..
HDB row 7942 checking against amenity df..
HDB row 7943 checking against amenity df..
HDB row 7944 checking against amenity df..
HDB row 7945 checking against amenity df..
HDB row 7946 checking against amenity df..
HDB row 7947 checking against amenity df..
HDB row 7948 checking against amenity df..
HDB row 7949 checking against amenity df..
HDB row 7950 checking against amenity df..
HDB row 7951 checking against amenity df..
HDB row 7952 checking against amenity df..
HDB row 7953 checking against amenity df..
HDB row 7954 checking against amenity df..
HDB row 7955 checking against amenity df..
HDB row 7956 checking against amenity df..
HDB row 7957 checking against amenity df..
HDB row 7958 checking against amenity df..
HDB row 7959 checking against amenity df..
HDB row 796

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,...,latitude,year_transacted,age,resale_price_inflate,psf,floor_area_sqf,nearest_mrt_station_dist,nearest_mrt_station,nearest_primary_school_dist,nearest_primary_school
0,2011-10-01,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,01 TO 03,60.0,Improved,1986,283000.0,...,1.375097,2011,35,311671.519213,482.587661,645.8340,0.420801,MAYFLOWER,0.232987,MAYFLOWER PRIMARY SCHOOL JAN 2019 - DEC 2021
1,2011-10-01,ANG MO KIO,3 ROOM,331,ANG MO KIO AVE 1,10 TO 12,68.0,New Generation,1981,343000.0,...,1.362111,2011,40,377750.286537,516.090940,731.9452,0.880294,ANG MO KIO,0.387262,TECK GHEE PRIMARY SCHOOL
2,2011-10-01,ANG MO KIO,3 ROOM,441,ANG MO KIO AVE 10,04 TO 06,67.0,New Generation,1979,333000.0,...,1.366050,2011,42,366737.158650,508.522834,721.1813,0.670305,ANG MO KIO,0.360764,TECK GHEE PRIMARY SCHOOL
3,2011-10-01,ANG MO KIO,3 ROOM,442,ANG MO KIO AVE 10,10 TO 12,82.0,New Generation,1979,399000.0,...,1.365653,2011,42,439423.802707,497.851788,882.6398,0.790373,ANG MO KIO,0.475436,TECK GHEE PRIMARY SCHOOL
4,2011-10-01,ANG MO KIO,3 ROOM,435,ANG MO KIO AVE 10,04 TO 06,82.0,New Generation,1979,383000.0,...,1.367704,2011,42,421802.798087,477.887807,882.6398,0.574072,ANG MO KIO,0.43194,TECK GHEE PRIMARY SCHOOL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8106,2012-02-01,YISHUN,5 ROOM,212,YISHUN ST 21,10 TO 12,121.0,Improved,1985,476888.0,...,1.431783,2012,36,502219.598563,385.601427,1302.4319,0.375451,YISHUN,0.19111,XISHAN PRIMARY SCHOOL
8107,2012-02-01,YISHUN,5 ROOM,758,YISHUN ST 72,01 TO 03,122.0,Improved,1986,490000.0,...,1.426030,2012,35,516028.088977,392.955939,1313.1958,0.388479,YISHUN,0.465217,JIEMIN PRIMARY SCHOOL
8108,2012-02-01,YISHUN,5 ROOM,873,YISHUN ST 81,01 TO 03,122.0,Improved,1988,488000.0,...,1.414695,2012,33,513921.851879,391.352037,1313.1958,0.533236,KHATIB,0.272357,NAVAL BASE PRIMARY SCHOOL
8109,2012-02-01,YISHUN,EXECUTIVE,664,YISHUN AVE 4,07 TO 09,181.0,Apartment,1992,705000.0,...,1.420200,2012,29,742448.576997,381.081749,1948.2659,0.95306,KHATIB,0.08718,NORTHLAND PRIMARY SCHOOL


### Add nearest secondary school to HDB df

In [31]:
sec_sch = pd.read_csv('../datasets/amenities/secondary_school.csv')

In [40]:
sec_sch

,secondary_school,longitude,latitude
0,ZHONGHUA SECONDARY SCHOOL,103.869430,1.348407
1,ZHENGHUA SECONDARY SCHOOL,103.765511,1.388366
2,YUYING SECONDARY SCHOOL,103.890195,1.357136
3,YUSOF ISHAK SECONDARY SCHOOL,103.760033,1.342339
4,YUHUA SECONDARY SCHOOL,103.722687,1.347186
5,YUAN CHING SECONDARY SCHOOL,103.720899,1.342006
6,JURONG SECONDARY SCHOOL,103.723953,1.330020
7,OUTRAM SECONDARY SCHOOL,103.837127,1.286514
8,YISHUN TOWN SECONDARY SCHOOL,103.838094,1.432322
9,ORCHID PARK SECONDARY SCHOOL,103.837859,1.415320


In [25]:
#get_distance_km_amenity(df, sec_sch, 'secondary_school')

![](images/get_dist.jpg)

In [26]:
#get_distance_km_amenity(df2, sec_sch, 'secondary_school')

![](images/get_dist.jpg)

In [27]:
#get_distance_km_amenity(df3, sec_sch, 'secondary_school')

![](images/get_dist.jpg)

In [42]:
get_distance_km_amenity(df4, sec_sch, 'secondary_school')

C:\Users\caspe\anaconda3\envs\capstone\lib\site-packages\ipykernel_launcher.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\caspe\anaconda3\envs\capstone\lib\site-packages\ipykernel_launcher.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


HDB row 0 checking against amenity df..
HDB row 1 checking against amenity df..
HDB row 2 checking against amenity df..
HDB row 3 checking against amenity df..
HDB row 4 checking against amenity df..
HDB row 5 checking against amenity df..
HDB row 6 checking against amenity df..
HDB row 7 checking against amenity df..
HDB row 8 checking against amenity df..
HDB row 9 checking against amenity df..
HDB row 10 checking against amenity df..
HDB row 11 checking against amenity df..
HDB row 12 checking against amenity df..
HDB row 13 checking against amenity df..
HDB row 14 checking against amenity df..
HDB row 15 checking against amenity df..
HDB row 16 checking against amenity df..
HDB row 17 checking against amenity df..
HDB row 18 checking against amenity df..
HDB row 19 checking against amenity df..
HDB row 20 checking against amenity df..
HDB row 21 checking against amenity df..
HDB row 22 checking against amenity df..
HDB row 23 checking against amenity df..
HDB row 24 checking agains

HDB row 198 checking against amenity df..
HDB row 199 checking against amenity df..
HDB row 200 checking against amenity df..
HDB row 201 checking against amenity df..
HDB row 202 checking against amenity df..
HDB row 203 checking against amenity df..
HDB row 204 checking against amenity df..
HDB row 205 checking against amenity df..
HDB row 206 checking against amenity df..
HDB row 207 checking against amenity df..
HDB row 208 checking against amenity df..
HDB row 209 checking against amenity df..
HDB row 210 checking against amenity df..
HDB row 211 checking against amenity df..
HDB row 212 checking against amenity df..
HDB row 213 checking against amenity df..
HDB row 214 checking against amenity df..
HDB row 215 checking against amenity df..
HDB row 216 checking against amenity df..
HDB row 217 checking against amenity df..
HDB row 218 checking against amenity df..
HDB row 219 checking against amenity df..
HDB row 220 checking against amenity df..
HDB row 221 checking against ameni

HDB row 394 checking against amenity df..
HDB row 395 checking against amenity df..
HDB row 396 checking against amenity df..
HDB row 397 checking against amenity df..
HDB row 398 checking against amenity df..
HDB row 399 checking against amenity df..
HDB row 400 checking against amenity df..
HDB row 401 checking against amenity df..
HDB row 402 checking against amenity df..
HDB row 403 checking against amenity df..
HDB row 404 checking against amenity df..
HDB row 405 checking against amenity df..
HDB row 406 checking against amenity df..
HDB row 407 checking against amenity df..
HDB row 408 checking against amenity df..
HDB row 409 checking against amenity df..
HDB row 410 checking against amenity df..
HDB row 411 checking against amenity df..
HDB row 412 checking against amenity df..
HDB row 413 checking against amenity df..
HDB row 414 checking against amenity df..
HDB row 415 checking against amenity df..
HDB row 416 checking against amenity df..
HDB row 417 checking against ameni

HDB row 595 checking against amenity df..
HDB row 596 checking against amenity df..
HDB row 597 checking against amenity df..
HDB row 598 checking against amenity df..
HDB row 599 checking against amenity df..
HDB row 600 checking against amenity df..
HDB row 601 checking against amenity df..
HDB row 602 checking against amenity df..
HDB row 603 checking against amenity df..
HDB row 604 checking against amenity df..
HDB row 605 checking against amenity df..
HDB row 606 checking against amenity df..
HDB row 607 checking against amenity df..
HDB row 608 checking against amenity df..
HDB row 609 checking against amenity df..
HDB row 610 checking against amenity df..
HDB row 611 checking against amenity df..
HDB row 612 checking against amenity df..
HDB row 613 checking against amenity df..
HDB row 614 checking against amenity df..
HDB row 615 checking against amenity df..
HDB row 616 checking against amenity df..
HDB row 617 checking against amenity df..
HDB row 618 checking against ameni

...

HDB row 7400 checking against amenity df..
HDB row 7401 checking against amenity df..
HDB row 7402 checking against amenity df..
HDB row 7403 checking against amenity df..
HDB row 7404 checking against amenity df..
HDB row 7405 checking against amenity df..
HDB row 7406 checking against amenity df..
HDB row 7407 checking against amenity df..
HDB row 7408 checking against amenity df..
HDB row 7409 checking against amenity df..
HDB row 7410 checking against amenity df..
HDB row 7411 checking against amenity df..
HDB row 7412 checking against amenity df..
HDB row 7413 checking against amenity df..
HDB row 7414 checking against amenity df..
HDB row 7415 checking against amenity df..
HDB row 7416 checking against amenity df..
HDB row 7417 checking against amenity df..
HDB row 7418 checking against amenity df..
HDB row 7419 checking against amenity df..
HDB row 7420 checking against amenity df..
HDB row 7421 checking against amenity df..
HDB row 7422 checking against amenity df..
HDB row 742

HDB row 7593 checking against amenity df..
HDB row 7594 checking against amenity df..
HDB row 7595 checking against amenity df..
HDB row 7596 checking against amenity df..
HDB row 7597 checking against amenity df..
HDB row 7598 checking against amenity df..
HDB row 7599 checking against amenity df..
HDB row 7600 checking against amenity df..
HDB row 7601 checking against amenity df..
HDB row 7602 checking against amenity df..
HDB row 7603 checking against amenity df..
HDB row 7604 checking against amenity df..
HDB row 7605 checking against amenity df..
HDB row 7606 checking against amenity df..
HDB row 7607 checking against amenity df..
HDB row 7608 checking against amenity df..
HDB row 7609 checking against amenity df..
HDB row 7610 checking against amenity df..
HDB row 7611 checking against amenity df..
HDB row 7612 checking against amenity df..
HDB row 7613 checking against amenity df..
HDB row 7614 checking against amenity df..
HDB row 7615 checking against amenity df..
HDB row 761

HDB row 7785 checking against amenity df..
HDB row 7786 checking against amenity df..
HDB row 7787 checking against amenity df..
HDB row 7788 checking against amenity df..
HDB row 7789 checking against amenity df..
HDB row 7790 checking against amenity df..
HDB row 7791 checking against amenity df..
HDB row 7792 checking against amenity df..
HDB row 7793 checking against amenity df..
HDB row 7794 checking against amenity df..
HDB row 7795 checking against amenity df..
HDB row 7796 checking against amenity df..
HDB row 7797 checking against amenity df..
HDB row 7798 checking against amenity df..
HDB row 7799 checking against amenity df..
HDB row 7800 checking against amenity df..
HDB row 7801 checking against amenity df..
HDB row 7802 checking against amenity df..
HDB row 7803 checking against amenity df..
HDB row 7804 checking against amenity df..
HDB row 7805 checking against amenity df..
HDB row 7806 checking against amenity df..
HDB row 7807 checking against amenity df..
HDB row 780

HDB row 7981 checking against amenity df..
HDB row 7982 checking against amenity df..
HDB row 7983 checking against amenity df..
HDB row 7984 checking against amenity df..
HDB row 7985 checking against amenity df..
HDB row 7986 checking against amenity df..
HDB row 7987 checking against amenity df..
HDB row 7988 checking against amenity df..
HDB row 7989 checking against amenity df..
HDB row 7990 checking against amenity df..
HDB row 7991 checking against amenity df..
HDB row 7992 checking against amenity df..
HDB row 7993 checking against amenity df..
HDB row 7994 checking against amenity df..
HDB row 7995 checking against amenity df..
HDB row 7996 checking against amenity df..
HDB row 7997 checking against amenity df..
HDB row 7998 checking against amenity df..
HDB row 7999 checking against amenity df..
HDB row 8000 checking against amenity df..
HDB row 8001 checking against amenity df..
HDB row 8002 checking against amenity df..
HDB row 8003 checking against amenity df..
HDB row 800

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,...,age,resale_price_inflate,psf,floor_area_sqf,nearest_mrt_station_dist,nearest_mrt_station,nearest_primary_school_dist,nearest_primary_school,nearest_secondary_school_dist,nearest_secondary_school
0,2011-10-01,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,01 TO 03,60.0,Improved,1986,283000.0,...,35,311671.519213,482.587661,645.8340,0.420801,MAYFLOWER,0.232987,MAYFLOWER PRIMARY SCHOOL JAN 2019 - DEC 2021,0.595743,YIO CHU KANG SECONDARY SCHOOL
1,2011-10-01,ANG MO KIO,3 ROOM,331,ANG MO KIO AVE 1,10 TO 12,68.0,New Generation,1981,343000.0,...,40,377750.286537,516.090940,731.9452,0.880294,ANG MO KIO,0.387262,TECK GHEE PRIMARY SCHOOL,0.475333,DEYI SECONDARY SCHOOL
2,2011-10-01,ANG MO KIO,3 ROOM,441,ANG MO KIO AVE 10,04 TO 06,67.0,New Generation,1979,333000.0,...,42,366737.158650,508.522834,721.1813,0.670305,ANG MO KIO,0.360764,TECK GHEE PRIMARY SCHOOL,0.18052,DEYI SECONDARY SCHOOL
3,2011-10-01,ANG MO KIO,3 ROOM,442,ANG MO KIO AVE 10,10 TO 12,82.0,New Generation,1979,399000.0,...,42,439423.802707,497.851788,882.6398,0.790373,ANG MO KIO,0.475436,TECK GHEE PRIMARY SCHOOL,0.301163,DEYI SECONDARY SCHOOL
4,2011-10-01,ANG MO KIO,3 ROOM,435,ANG MO KIO AVE 10,04 TO 06,82.0,New Generation,1979,383000.0,...,42,421802.798087,477.887807,882.6398,0.574072,ANG MO KIO,0.43194,TECK GHEE PRIMARY SCHOOL,0.265444,DEYI SECONDARY SCHOOL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8106,2012-02-01,YISHUN,5 ROOM,212,YISHUN ST 21,10 TO 12,121.0,Improved,1985,476888.0,...,36,502219.598563,385.601427,1302.4319,0.375451,YISHUN,0.19111,XISHAN PRIMARY SCHOOL,0.094359,YISHUN TOWN SECONDARY SCHOOL
8107,2012-02-01,YISHUN,5 ROOM,758,YISHUN ST 72,01 TO 03,122.0,Improved,1986,490000.0,...,35,516028.088977,392.955939,1313.1958,0.388479,YISHUN,0.465217,JIEMIN PRIMARY SCHOOL,0.654995,YISHUN SECONDARY SCHOOL
8108,2012-02-01,YISHUN,5 ROOM,873,YISHUN ST 81,01 TO 03,122.0,Improved,1988,488000.0,...,33,513921.851879,391.352037,1313.1958,0.533236,KHATIB,0.272357,NAVAL BASE PRIMARY SCHOOL,0.122708,ORCHID PARK SECONDARY SCHOOL
8109,2012-02-01,YISHUN,EXECUTIVE,664,YISHUN AVE 4,07 TO 09,181.0,Apartment,1992,705000.0,...,29,742448.576997,381.081749,1948.2659,0.95306,KHATIB,0.08718,NORTHLAND PRIMARY SCHOOL,0.457969,NORTHBROOKS SECONDARY SCHOOL


### Add nearest mall to HDB df

In [33]:
mall = pd.read_csv('../datasets/amenities/mall.csv')

In [34]:
mall

,mall,longitude,latitude
0,100 AM,103.843488,1.274683
1,UOB 313@SOMERSET,103.837684,1.301385
2,APERIA,103.863930,1.310867
3,BALESTIER HILL SHOPPING CENTRE,103.843741,1.326163
4,BUGIS CUBE,103.855635,1.298141
5,BUGIS JUNCTION,103.855411,1.299113
6,BUGIS BACKPACKERS HOSTEL,103.855173,1.300952
7,CAPITOL PIAZZA,103.851262,1.293079
8,CATHAY CINELEISURE ORCHARD,103.836430,1.301521
9,Clarke Quay Central,NaN,NaN


Drop null rows as the OneMap API does not return any coordiantes regardless of spelling.

In [35]:
mall.dropna(inplace=True)

In [28]:
#get_distance_km_amenity(df, mall, 'mall')

![](images/get_dist.jpg)

In [29]:
#get_distance_km_amenity(df2, mall, 'mall')

![](images/get_dist.jpg)

In [30]:
#get_distance_km_amenity(df3, mall, 'mall')

![](images/get_dist.jpg)

In [46]:
get_distance_km_amenity(df4, mall, 'mall')

C:\Users\caspe\anaconda3\envs\capstone\lib\site-packages\ipykernel_launcher.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\caspe\anaconda3\envs\capstone\lib\site-packages\ipykernel_launcher.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


HDB row 0 checking against amenity df..
HDB row 1 checking against amenity df..
HDB row 2 checking against amenity df..
HDB row 3 checking against amenity df..
HDB row 4 checking against amenity df..
HDB row 5 checking against amenity df..
HDB row 6 checking against amenity df..
HDB row 7 checking against amenity df..
HDB row 8 checking against amenity df..
HDB row 9 checking against amenity df..
HDB row 10 checking against amenity df..
HDB row 11 checking against amenity df..
HDB row 12 checking against amenity df..
HDB row 13 checking against amenity df..
HDB row 14 checking against amenity df..
HDB row 15 checking against amenity df..
HDB row 16 checking against amenity df..
HDB row 17 checking against amenity df..
HDB row 18 checking against amenity df..
HDB row 19 checking against amenity df..
HDB row 20 checking against amenity df..
HDB row 21 checking against amenity df..
HDB row 22 checking against amenity df..
HDB row 23 checking against amenity df..
HDB row 24 checking agains

HDB row 200 checking against amenity df..
HDB row 201 checking against amenity df..
HDB row 202 checking against amenity df..
HDB row 203 checking against amenity df..
HDB row 204 checking against amenity df..
HDB row 205 checking against amenity df..
HDB row 206 checking against amenity df..
HDB row 207 checking against amenity df..
HDB row 208 checking against amenity df..
HDB row 209 checking against amenity df..
HDB row 210 checking against amenity df..
HDB row 211 checking against amenity df..
HDB row 212 checking against amenity df..
HDB row 213 checking against amenity df..
HDB row 214 checking against amenity df..
HDB row 215 checking against amenity df..
HDB row 216 checking against amenity df..
HDB row 217 checking against amenity df..
HDB row 218 checking against amenity df..
HDB row 219 checking against amenity df..
HDB row 220 checking against amenity df..
HDB row 221 checking against amenity df..
HDB row 222 checking against amenity df..
HDB row 223 checking against ameni

HDB row 396 checking against amenity df..
HDB row 397 checking against amenity df..
HDB row 398 checking against amenity df..
HDB row 399 checking against amenity df..
HDB row 400 checking against amenity df..
HDB row 401 checking against amenity df..
HDB row 402 checking against amenity df..
HDB row 403 checking against amenity df..
HDB row 404 checking against amenity df..
HDB row 405 checking against amenity df..
HDB row 406 checking against amenity df..
HDB row 407 checking against amenity df..
HDB row 408 checking against amenity df..
HDB row 409 checking against amenity df..
HDB row 410 checking against amenity df..
HDB row 411 checking against amenity df..
HDB row 412 checking against amenity df..
HDB row 413 checking against amenity df..
HDB row 414 checking against amenity df..
HDB row 415 checking against amenity df..
HDB row 416 checking against amenity df..
HDB row 417 checking against amenity df..
HDB row 418 checking against amenity df..
HDB row 419 checking against ameni

HDB row 594 checking against amenity df..
HDB row 595 checking against amenity df..
HDB row 596 checking against amenity df..
HDB row 597 checking against amenity df..
HDB row 598 checking against amenity df..
HDB row 599 checking against amenity df..
HDB row 600 checking against amenity df..
HDB row 601 checking against amenity df..
HDB row 602 checking against amenity df..
HDB row 603 checking against amenity df..
HDB row 604 checking against amenity df..
HDB row 605 checking against amenity df..
HDB row 606 checking against amenity df..
HDB row 607 checking against amenity df..
HDB row 608 checking against amenity df..
HDB row 609 checking against amenity df..
HDB row 610 checking against amenity df..
HDB row 611 checking against amenity df..
HDB row 612 checking against amenity df..
HDB row 613 checking against amenity df..
HDB row 614 checking against amenity df..
HDB row 615 checking against amenity df..
HDB row 616 checking against amenity df..
HDB row 617 checking against ameni

...

HDB row 7382 checking against amenity df..
HDB row 7383 checking against amenity df..
HDB row 7384 checking against amenity df..
HDB row 7385 checking against amenity df..
HDB row 7386 checking against amenity df..
HDB row 7387 checking against amenity df..
HDB row 7388 checking against amenity df..
HDB row 7389 checking against amenity df..
HDB row 7390 checking against amenity df..
HDB row 7391 checking against amenity df..
HDB row 7392 checking against amenity df..
HDB row 7393 checking against amenity df..
HDB row 7394 checking against amenity df..
HDB row 7395 checking against amenity df..
HDB row 7396 checking against amenity df..
HDB row 7397 checking against amenity df..
HDB row 7398 checking against amenity df..
HDB row 7399 checking against amenity df..
HDB row 7400 checking against amenity df..
HDB row 7401 checking against amenity df..
HDB row 7402 checking against amenity df..
HDB row 7403 checking against amenity df..
HDB row 7404 checking against amenity df..
HDB row 740

HDB row 7575 checking against amenity df..
HDB row 7576 checking against amenity df..
HDB row 7577 checking against amenity df..
HDB row 7578 checking against amenity df..
HDB row 7579 checking against amenity df..
HDB row 7580 checking against amenity df..
HDB row 7581 checking against amenity df..
HDB row 7582 checking against amenity df..
HDB row 7583 checking against amenity df..
HDB row 7584 checking against amenity df..
HDB row 7585 checking against amenity df..
HDB row 7586 checking against amenity df..
HDB row 7587 checking against amenity df..
HDB row 7588 checking against amenity df..
HDB row 7589 checking against amenity df..
HDB row 7590 checking against amenity df..
HDB row 7591 checking against amenity df..
HDB row 7592 checking against amenity df..
HDB row 7593 checking against amenity df..
HDB row 7594 checking against amenity df..
HDB row 7595 checking against amenity df..
HDB row 7596 checking against amenity df..
HDB row 7597 checking against amenity df..
HDB row 759

HDB row 7769 checking against amenity df..
HDB row 7770 checking against amenity df..
HDB row 7771 checking against amenity df..
HDB row 7772 checking against amenity df..
HDB row 7773 checking against amenity df..
HDB row 7774 checking against amenity df..
HDB row 7775 checking against amenity df..
HDB row 7776 checking against amenity df..
HDB row 7777 checking against amenity df..
HDB row 7778 checking against amenity df..
HDB row 7779 checking against amenity df..
HDB row 7780 checking against amenity df..
HDB row 7781 checking against amenity df..
HDB row 7782 checking against amenity df..
HDB row 7783 checking against amenity df..
HDB row 7784 checking against amenity df..
HDB row 7785 checking against amenity df..
HDB row 7786 checking against amenity df..
HDB row 7787 checking against amenity df..
HDB row 7788 checking against amenity df..
HDB row 7789 checking against amenity df..
HDB row 7790 checking against amenity df..
HDB row 7791 checking against amenity df..
HDB row 779

HDB row 7961 checking against amenity df..
HDB row 7962 checking against amenity df..
HDB row 7963 checking against amenity df..
HDB row 7964 checking against amenity df..
HDB row 7965 checking against amenity df..
HDB row 7966 checking against amenity df..
HDB row 7967 checking against amenity df..
HDB row 7968 checking against amenity df..
HDB row 7969 checking against amenity df..
HDB row 7970 checking against amenity df..
HDB row 7971 checking against amenity df..
HDB row 7972 checking against amenity df..
HDB row 7973 checking against amenity df..
HDB row 7974 checking against amenity df..
HDB row 7975 checking against amenity df..
HDB row 7976 checking against amenity df..
HDB row 7977 checking against amenity df..
HDB row 7978 checking against amenity df..
HDB row 7979 checking against amenity df..
HDB row 7980 checking against amenity df..
HDB row 7981 checking against amenity df..
HDB row 7982 checking against amenity df..
HDB row 7983 checking against amenity df..
HDB row 798

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,...,psf,floor_area_sqf,nearest_mrt_station_dist,nearest_mrt_station,nearest_primary_school_dist,nearest_primary_school,nearest_secondary_school_dist,nearest_secondary_school,nearest_mall_dist,nearest_mall
0,2011-10-01,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,01 TO 03,60.0,Improved,1986,283000.0,...,482.587661,645.8340,0.420801,MAYFLOWER,0.232987,MAYFLOWER PRIMARY SCHOOL JAN 2019 - DEC 2021,0.595743,YIO CHU KANG SECONDARY SCHOOL,0.990781,BROADWAY PLAZA
1,2011-10-01,ANG MO KIO,3 ROOM,331,ANG MO KIO AVE 1,10 TO 12,68.0,New Generation,1981,343000.0,...,516.090940,731.9452,0.880294,ANG MO KIO,0.387262,TECK GHEE PRIMARY SCHOOL,0.475333,DEYI SECONDARY SCHOOL,0.848568,AMK HUB
2,2011-10-01,ANG MO KIO,3 ROOM,441,ANG MO KIO AVE 10,04 TO 06,67.0,New Generation,1979,333000.0,...,508.522834,721.1813,0.670305,ANG MO KIO,0.360764,TECK GHEE PRIMARY SCHOOL,0.18052,DEYI SECONDARY SCHOOL,0.733697,AMK HUB
3,2011-10-01,ANG MO KIO,3 ROOM,442,ANG MO KIO AVE 10,10 TO 12,82.0,New Generation,1979,399000.0,...,497.851788,882.6398,0.790373,ANG MO KIO,0.475436,TECK GHEE PRIMARY SCHOOL,0.301163,DEYI SECONDARY SCHOOL,0.858353,AMK HUB
4,2011-10-01,ANG MO KIO,3 ROOM,435,ANG MO KIO AVE 10,04 TO 06,82.0,New Generation,1979,383000.0,...,477.887807,882.6398,0.574072,ANG MO KIO,0.43194,TECK GHEE PRIMARY SCHOOL,0.265444,DEYI SECONDARY SCHOOL,0.664854,AMK HUB
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8106,2012-02-01,YISHUN,5 ROOM,212,YISHUN ST 21,10 TO 12,121.0,Improved,1985,476888.0,...,385.601427,1302.4319,0.375451,YISHUN,0.19111,XISHAN PRIMARY SCHOOL,0.094359,YISHUN TOWN SECONDARY SCHOOL,0.296512,NORTHPOINT CITY
8107,2012-02-01,YISHUN,5 ROOM,758,YISHUN ST 72,01 TO 03,122.0,Improved,1986,490000.0,...,392.955939,1313.1958,0.388479,YISHUN,0.465217,JIEMIN PRIMARY SCHOOL,0.654995,YISHUN SECONDARY SCHOOL,0.441306,NORTHPOINT CITY
8108,2012-02-01,YISHUN,5 ROOM,873,YISHUN ST 81,01 TO 03,122.0,Improved,1988,488000.0,...,391.352037,1313.1958,0.533236,KHATIB,0.272357,NAVAL BASE PRIMARY SCHOOL,0.122708,ORCHID PARK SECONDARY SCHOOL,0.601607,WISTERIA MALL
8109,2012-02-01,YISHUN,EXECUTIVE,664,YISHUN AVE 4,07 TO 09,181.0,Apartment,1992,705000.0,...,381.081749,1948.2659,0.95306,KHATIB,0.08718,NORTHLAND PRIMARY SCHOOL,0.457969,NORTHBROOKS SECONDARY SCHOOL,0.250862,WISTERIA MALL


### Add dist to CBD to HDB df

[URA](https://www.ura.gov.sg/Corporate/Guidelines/Urban-Design/-/media/67DDEA6B29AE426DA7E471DB1375E638.ashx) defines the CBD of Singapore to be the area which spans from Raffles Place along Shenton Way/Robinson Road/Cecil Street to the Tanjong Pagar and Anson subzones. It also extends to Marina Bay, including the Central and Bayfront subzones.

In this case, we will take the longitudes and latitudes of Raffles Place MRT station as the centre of CBD.

Raffles Place Mrt Station: lat = 1.28393326234538, long = 103.851463066212

In [47]:
def get_distance_km_CBD(hdb_df):
    
    '''
    calculate the distance to CBD,
    return CBD distance
    '''
    hdb_df['dist_cbd'] = 0
    hdb_df['dist_cbd'] = hdb_df['dist_cbd'].astype(float)
    
    row = 0
    
    cbd_lat = 1.28393326234538
    cbd_long = 103.851463066212
    
    lat1 = math.radians(cbd_lat)
    lon1 = math.radians(cbd_long)

    for long, lat in zip(hdb_df['longitude'], hdb_df['latitude']): 

        lat2 = math.radians(lat)
        lon2 = math.radians(long)
        
        dlon = lon2 - lon1
        dlat = lat2 - lat1

        # Using Haversine formula
        a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
        c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
        dist = 6373 * c

        hdb_df['dist_cbd'][row] = dist

        row += 1   
        
    return hdb_df
    

In [31]:
#get_distance_km_CBD(df)

![](images/cbd(df).jpg)

In [32]:
#get_distance_km_CBD(df2)

![](images/cbd(df2).jpg)

In [33]:
#get_distance_km_CBD(df3)

![](images/cbd(df3).jpg)

In [34]:
#get_distance_km_CBD(df4)

![](images/cbd(df4).jpg)

# Numeralize storey_range column

In [49]:
def storey_value(row):
    
    '''takes the average of the storey_range column and returns a new column avg_storey'''
    
    row['avg_storey'] = (int(row['storey_range'].split()[0]) + int(row['storey_range'].split()[2])) / 2
    return row

In [43]:
df = df.apply(storey_value, axis=1)
df

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,...,nearest_mrt_station_dist,nearest_mrt_station,nearest_primary_school_dist,nearest_primary_school,nearest_secondary_school_dist,nearest_secondary_school,nearest_mall_dist,nearest_mall,dist_cbd,avg_storey
0,2015-01-01,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,07 TO 09,60.0,Improved,1986,255000.0,...,0.420801,MAYFLOWER,0.232987,MAYFLOWER PRIMARY SCHOOL JAN 2019 - DEC 2021,0.595743,YIO CHU KANG SECONDARY SCHOOL,0.990781,BROADWAY PLAZA,10.256373,8.0
1,2015-01-01,ANG MO KIO,3 ROOM,541,ANG MO KIO AVE 10,01 TO 03,68.0,New Generation,1981,275000.0,...,0.807154,ANG MO KIO,0.431101,JING SHAN PRIMARY SCHOOL,0.454650,ANDERSON SECONDARY SCHOOL,0.897583,JUBILEE SQUARE,10.020151,2.0
2,2015-01-01,ANG MO KIO,3 ROOM,163,ANG MO KIO AVE 4,01 TO 03,69.0,New Generation,1980,285000.0,...,0.292697,MAYFLOWER,0.390335,MAYFLOWER PRIMARY SCHOOL JAN 2019 - DEC 2021,0.660947,YIO CHU KANG SECONDARY SCHOOL,0.883615,BROADWAY PLAZA,10.077275,2.0
3,2015-01-01,ANG MO KIO,3 ROOM,446,ANG MO KIO AVE 10,01 TO 03,68.0,New Generation,1979,290000.0,...,0.688610,ANG MO KIO,0.545725,TECK GHEE PRIMARY SCHOOL,0.369088,DEYI SECONDARY SCHOOL,0.786100,AMK HUB,9.334189,2.0
4,2015-01-01,ANG MO KIO,3 ROOM,557,ANG MO KIO AVE 10,07 TO 09,68.0,New Generation,1980,290000.0,...,0.928669,ANG MO KIO,0.626791,JING SHAN PRIMARY SCHOOL,0.750803,ANDERSON SECONDARY SCHOOL,1.059116,AMK HUB,9.778927,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37148,2016-12-01,YISHUN,5 ROOM,297,YISHUN ST 20,13 TO 15,112.0,Improved,2000,488000.0,...,0.706537,YISHUN,0.302963,YISHUN PRIMARY SCHOOL,0.489373,YISHUN TOWN SECONDARY SCHOOL,0.678345,NORTHPOINT CITY,16.984775,14.0
37149,2016-12-01,YISHUN,5 ROOM,838,YISHUN ST 81,01 TO 03,122.0,Improved,1987,455000.0,...,0.124889,KHATIB,0.360926,PEIYING PRIMARY SCHOOL,0.431325,NAVAL BASE SECONDARY SCHOOL,0.873969,WISTERIA MALL,14.870542,2.0
37150,2016-12-01,YISHUN,EXECUTIVE,664,YISHUN AVE 4,10 TO 12,181.0,Apartment,1992,778000.0,...,0.953060,KHATIB,0.087180,NORTHLAND PRIMARY SCHOOL,0.457969,NORTHBROOKS SECONDARY SCHOOL,0.250862,WISTERIA MALL,15.200869,11.0
37151,2016-12-01,YISHUN,EXECUTIVE,325,YISHUN CTRL,01 TO 03,146.0,Maisonette,1988,575000.0,...,0.794331,YISHUN,0.343155,HUAMIN PRIMARY SCHOOL,0.566170,YISHUN TOWN SECONDARY SCHOOL,0.428650,DBS JUNCTION NINE,16.195459,2.0


In [25]:
df2 = df2.apply(storey_value, axis=1)
df2

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,...,psf,floor_area_sqf,nearest_mrt_station_dist,nearest_mrt_station,nearest_primary_school_dist,nearest_primary_school,nearest_mall_dist,nearest_mall,dist_cbd,avg_storey
0,2012-03-01,ANG MO KIO,2 ROOM,172,ANG MO KIO AVE 4,06 TO 10,45.0,Improved,1986,250000.0,...,543.544496,484.3755,0.359578,MAYFLOWER,0.177288,MAYFLOWER PRIMARY SCHOOL JAN 2019 - DEC 2021,1.098555,BROADWAY PLAZA,10.232201,8.0
1,2012-03-01,ANG MO KIO,2 ROOM,510,ANG MO KIO AVE 8,01 TO 05,44.0,Improved,1980,265000.0,...,589.251647,473.6116,0.389503,ANG MO KIO,0.364117,JING SHAN PRIMARY SCHOOL,0.226192,JUBILEE SQUARE,9.955021,3.0
2,2012-03-01,ANG MO KIO,3 ROOM,610,ANG MO KIO AVE 4,06 TO 10,68.0,New Generation,1980,315000.0,...,453.220190,731.9452,0.695344,YIO CHU KANG,0.519123,MAYFLOWER PRIMARY SCHOOL JAN 2019 - DEC 2021,1.121827,BROADWAY PLAZA,10.705974,8.0
3,2012-03-01,ANG MO KIO,3 ROOM,474,ANG MO KIO AVE 10,01 TO 05,67.0,New Generation,1984,320000.0,...,467.286015,721.1813,1.233422,ANG MO KIO,0.555781,TOWNSVILLE PRIMARY SCHOOL,0.836564,MYVILLAGE AT SERANGOON GARDEN,8.797868,3.0
4,2012-03-01,ANG MO KIO,3 ROOM,604,ANG MO KIO AVE 5,06 TO 10,67.0,New Generation,1980,321000.0,...,468.746284,721.1813,0.627870,LENTOR,0.414254,MAYFLOWER PRIMARY SCHOOL JAN 2019 - DEC 2021,1.416917,BROADWAY PLAZA,10.808713,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52165,2014-12-01,YISHUN,5 ROOM,816,YISHUN ST 81,10 TO 12,122.0,Improved,1988,580000.0,...,449.805272,1313.1958,0.625761,KHATIB,0.690544,PEIYING PRIMARY SCHOOL,1.115615,WISTERIA MALL,14.361022,11.0
52166,2014-12-01,YISHUN,EXECUTIVE,325,YISHUN CTRL,10 TO 12,146.0,Maisonette,1988,540000.0,...,349.942977,1571.5294,0.794331,YISHUN,0.343155,HUAMIN PRIMARY SCHOOL,0.428650,DBS JUNCTION NINE,16.195459,11.0
52167,2014-12-01,YISHUN,EXECUTIVE,618,YISHUN RING RD,07 TO 09,164.0,Apartment,1992,738000.0,...,425.763956,1765.2796,0.338108,KHATIB,0.437143,NAVAL BASE PRIMARY SCHOOL,0.624955,WISTERIA MALL,15.096074,8.0
52168,2014-12-01,YISHUN,EXECUTIVE,277,YISHUN ST 22,07 TO 09,152.0,Maisonette,1985,592000.0,...,368.497459,1636.1128,0.968322,YISHUN,0.501772,XISHAN PRIMARY SCHOOL,0.748267,DBS JUNCTION NINE,17.203143,8.0


In [72]:
df3 = df3.apply(storey_value, axis=1)
df3

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,...,nearest_mrt_station_dist,nearest_mrt_station,nearest_primary_school_dist,nearest_primary_school,nearest_secondary_school_dist,nearest_secondary_school,nearest_mall_dist,nearest_mall,dist_cbd,avg_storey
0,2017-01-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,232000.0,...,1.004311,ANG MO KIO,0.219416,TOWNSVILLE PRIMARY SCHOOL,0.468208,DEYI SECONDARY SCHOOL,1.017604,AMK HUB,8.688013,11.0
1,2017-01-01,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67.0,New Generation,1978,250000.0,...,0.166825,MAYFLOWER,0.257720,ANG MO KIO PRIMARY SCHOOL,0.415571,MAYFLOWER SECONDARY SCHOOL,0.893638,BROADWAY PLAZA,9.793586,2.0
2,2017-01-01,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,262000.0,...,0.535285,LENTOR,0.509820,MAYFLOWER PRIMARY SCHOOL JAN 2019 - DEC 2021,0.811735,YIO CHU KANG SECONDARY SCHOOL,1.528502,BROADWAY PLAZA,10.912103,2.0
3,2017-01-01,ANG MO KIO,3 ROOM,465,ANG MO KIO AVE 10,04 TO 06,68.0,New Generation,1980,265000.0,...,0.945827,ANG MO KIO,0.697650,TECK GHEE PRIMARY SCHOOL,0.518150,DEYI SECONDARY SCHOOL,0.893179,MYVILLAGE AT SERANGOON GARDEN,9.172850,5.0
4,2017-01-01,ANG MO KIO,3 ROOM,601,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,265000.0,...,0.501308,LENTOR,0.549612,MAYFLOWER PRIMARY SCHOOL JAN 2019 - DEC 2021,0.850321,YIO CHU KANG SECONDARY SCHOOL,1.572398,BROADWAY PLAZA,10.952920,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109246,2021-10-01,YISHUN,4 ROOM,458,YISHUN AVE 11,10 TO 12,93.0,Model A,2013,468888.0,...,1.515562,KHATIB,0.588007,NORTHLAND PRIMARY SCHOOL,0.119025,NORTHBROOKS SECONDARY SCHOOL,0.626715,WISTERIA MALL,15.220129,11.0
109247,2021-10-01,YISHUN,4 ROOM,621,YISHUN RING RD,07 TO 09,107.0,Model A,1988,495000.0,...,0.283930,KHATIB,0.479379,NAVAL BASE PRIMARY SCHOOL,0.466738,NAVAL BASE SECONDARY SCHOOL,0.680923,WISTERIA MALL,15.095305,8.0
109248,2021-10-01,YISHUN,4 ROOM,512B,YISHUN ST 51,04 TO 06,93.0,Model A,2017,475000.0,...,1.058218,KHATIB,0.409630,NAVAL BASE PRIMARY SCHOOL,0.477817,ORCHID PARK SECONDARY SCHOOL,0.373018,WISTERIA MALL,14.583063,5.0
109249,2021-10-01,YISHUN,5 ROOM,342A,YISHUN RING RD,04 TO 06,117.0,Premium Apartment,2016,611888.0,...,1.059682,YISHUN,0.403899,HUAMIN PRIMARY SCHOOL,0.733886,YISHUN TOWN SECONDARY SCHOOL,0.416015,DBS JUNCTION NINE,16.331946,5.0


In [50]:
df4 = df4.apply(storey_value, axis=1)
df4

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,...,nearest_mrt_station_dist,nearest_mrt_station,nearest_primary_school_dist,nearest_primary_school,nearest_secondary_school_dist,nearest_secondary_school,nearest_mall_dist,nearest_mall,dist_cbd,avg_storey
0,2011-10-01,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,01 TO 03,60.0,Improved,1986,283000.0,...,0.420801,MAYFLOWER,0.232987,MAYFLOWER PRIMARY SCHOOL JAN 2019 - DEC 2021,0.595743,YIO CHU KANG SECONDARY SCHOOL,0.990781,BROADWAY PLAZA,10.256373,2.0
1,2011-10-01,ANG MO KIO,3 ROOM,331,ANG MO KIO AVE 1,10 TO 12,68.0,New Generation,1981,343000.0,...,0.880294,ANG MO KIO,0.387262,TECK GHEE PRIMARY SCHOOL,0.475333,DEYI SECONDARY SCHOOL,0.848568,AMK HUB,8.696086,11.0
2,2011-10-01,ANG MO KIO,3 ROOM,441,ANG MO KIO AVE 10,04 TO 06,67.0,New Generation,1979,333000.0,...,0.670305,ANG MO KIO,0.360764,TECK GHEE PRIMARY SCHOOL,0.180520,DEYI SECONDARY SCHOOL,0.733697,AMK HUB,9.138814,5.0
3,2011-10-01,ANG MO KIO,3 ROOM,442,ANG MO KIO AVE 10,10 TO 12,82.0,New Generation,1979,399000.0,...,0.790373,ANG MO KIO,0.475436,TECK GHEE PRIMARY SCHOOL,0.301163,DEYI SECONDARY SCHOOL,0.858353,AMK HUB,9.099361,11.0
4,2011-10-01,ANG MO KIO,3 ROOM,435,ANG MO KIO AVE 10,04 TO 06,82.0,New Generation,1979,383000.0,...,0.574072,ANG MO KIO,0.431940,TECK GHEE PRIMARY SCHOOL,0.265444,DEYI SECONDARY SCHOOL,0.664854,AMK HUB,9.322790,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8106,2012-02-01,YISHUN,5 ROOM,212,YISHUN ST 21,10 TO 12,121.0,Improved,1985,476888.0,...,0.375451,YISHUN,0.191110,XISHAN PRIMARY SCHOOL,0.094359,YISHUN TOWN SECONDARY SCHOOL,0.296512,NORTHPOINT CITY,16.519100,11.0
8107,2012-02-01,YISHUN,5 ROOM,758,YISHUN ST 72,01 TO 03,122.0,Improved,1986,490000.0,...,0.388479,YISHUN,0.465217,JIEMIN PRIMARY SCHOOL,0.654995,YISHUN SECONDARY SCHOOL,0.441306,NORTHPOINT CITY,15.920607,2.0
8108,2012-02-01,YISHUN,5 ROOM,873,YISHUN ST 81,01 TO 03,122.0,Improved,1988,488000.0,...,0.533236,KHATIB,0.272357,NAVAL BASE PRIMARY SCHOOL,0.122708,ORCHID PARK SECONDARY SCHOOL,0.601607,WISTERIA MALL,14.633841,2.0
8109,2012-02-01,YISHUN,EXECUTIVE,664,YISHUN AVE 4,07 TO 09,181.0,Apartment,1992,705000.0,...,0.953060,KHATIB,0.087180,NORTHLAND PRIMARY SCHOOL,0.457969,NORTHBROOKS SECONDARY SCHOOL,0.250862,WISTERIA MALL,15.200869,8.0


## Last check on HDB dfs

After adding in new features to our dfs, we will check for any null rows.

<b>Check on df</b>

In [44]:
df.isnull().sum()

month                             0
town                              0
flat_type                         0
block                             0
street_name                       0
storey_range                      0
floor_area_sqm                    0
flat_model                        0
lease_commence_date               0
resale_price                      0
longitude                        80
latitude                         80
year_transacted                   0
age                               0
resale_price_inflate              0
psf                               0
floor_area_sqf                    0
nearest_mrt_station_dist         80
nearest_mrt_station              80
nearest_primary_school_dist      80
nearest_primary_school           80
nearest_secondary_school_dist    80
nearest_secondary_school         80
nearest_mall_dist                80
nearest_mall                      0
dist_cbd                         80
avg_storey                        0
dtype: int64

In [48]:
df.shape

(37153, 27)

In [49]:
80/37153*100

0.21532581487363067

We note there were about 80 rows which we were unable to scrap for the geo coordinates of the HDB. This represents about 0.22% of the dataset for 2015-2016. Since it is a small amount, we will choose to drop these null rows.

In [50]:
# drop null rows
df.dropna(inplace=True)

In [51]:
# save to new csv file
df.to_csv('../datasets/final/hdb_2015_16 (22 Oct).csv', index=False)

<b>Check on df2</b>

In [27]:
df2.isnull().mean()

month                          0.000000
town                           0.000000
flat_type                      0.000000
block                          0.000000
street_name                    0.000000
storey_range                   0.000000
floor_area_sqm                 0.000000
flat_model                     0.000000
lease_commence_date            0.000000
resale_price                   0.000000
longitude                      0.002473
latitude                       0.002473
year_transacted                0.000000
age                            0.000000
resale_price_inflate           0.000000
psf                            0.000000
floor_area_sqf                 0.000000
nearest_mrt_station_dist       0.002473
nearest_mrt_station            0.002473
nearest_primary_school_dist    0.002473
nearest_primary_school         0.000000
nearest_mall_dist              0.002473
nearest_mall                   0.000000
dist_cbd                       0.002473
avg_storey                     0.000000


Similarly, there were some null rows which we were unable to scrap for the geo coordinates of the HDB. This represents about 0.25% of the dataset for 2015-2016. Since it is a small amount, we will choose to drop these null rows.

In [28]:
df2.dropna(inplace=True)

In [60]:
df2.shape

(52041, 27)

In [64]:
df2.to_csv('../datasets/final/hdb_2012_14 (24 Oct).csv', index=False)

<b>Check on df3</b>

In [73]:
df3.isnull().mean()

month                            0.000000
town                             0.000000
flat_type                        0.000000
block                            0.000000
street_name                      0.000000
storey_range                     0.000000
floor_area_sqm                   0.000000
flat_model                       0.000000
lease_commence_date              0.000000
resale_price                     0.000000
longitude                        0.002188
latitude                         0.002188
year_transacted                  0.000000
age                              0.000000
resale_price_inflate             0.000000
psf                              0.000000
floor_area_sqf                   0.000000
nearest_mrt_station_dist         0.002188
nearest_mrt_station              0.000000
nearest_primary_school_dist      0.002188
nearest_primary_school           0.000000
nearest_secondary_school_dist    0.002188
nearest_secondary_school         0.000000
nearest_mall_dist                0

Similarly, the null rows for df3 represents about 0.22% of the dataset for 2017-2021. Since it is a small amount, we will choose to drop these null rows.

In [78]:
df3.dropna(inplace=True)

In [79]:
df3.to_csv('../datasets/final/hdb_2017_21 (23 Oct).csv', index=False)

<b>Check on df4</b>

In [55]:
df4.isnull().mean()

month                            0.000000
town                             0.000000
flat_type                        0.000000
block                            0.000000
street_name                      0.000000
storey_range                     0.000000
floor_area_sqm                   0.000000
flat_model                       0.000000
lease_commence_date              0.000000
resale_price                     0.000000
longitude                        0.002712
latitude                         0.002712
year_transacted                  0.000000
age                              0.000000
resale_price_inflate             0.000000
psf                              0.000000
floor_area_sqf                   0.000000
nearest_mrt_station_dist         0.002712
nearest_mrt_station              0.000000
nearest_primary_school_dist      0.002712
nearest_primary_school           0.000000
nearest_secondary_school_dist    0.002712
nearest_secondary_school         0.000000
nearest_mall_dist                0

In [57]:
df4.dropna(inplace=True)

In [59]:
df4.to_csv('../datasets/final/hdb_2011_oct_2012_feb (24 Oct).csv', index=False)

Subsequently, while trying to visualize spatial data on Tableau, I realize that there were some HDBs (specifically from the address Kent Rd) which had their geocoordinates put incorrectly from the OneMap API. It returned several searches for the same address and the top search were wrong coordinates. Namely, the rows which were impacted were:

- Block 51 Kent Rd
- Block 52 Kent Rd
- Block 53 Kent Rd
- Block 54 Kent Rd

These transactions should have appeared closer to Farrer Park area but strangly appeared in the Pasir Panjang area on Tableau visualization. 

In [90]:
# read in dfs with all the geo coordinates plus newly engineered features
df = pd.read_csv('../datasets/final/hdb_2011_oct_2012_feb (24 Oct).csv')
df2 = pd.read_csv('../datasets/final/hdb_2012_14 (24 Oct).csv')
df3 = pd.read_csv('../datasets/final/hdb_2015_16 (22 Oct).csv')
df4 = pd.read_csv('../datasets/final/hdb_2017_21 (23 Oct).csv')

In [91]:
df = pd.concat([df, df2, df3, df4], axis=0)

In [92]:
df[df['street_name'] == 'KENT RD'].groupby('block')['street_name'].count()

block
51    13
52    18
53     4
54    15
Name: street_name, dtype: int64

Upon closer examination, there are 50 rows that we would need to replace their longitudes and latitudes accordingly.

The correct longitudes and latitudes for block 51 to 54 are:

- block 51 coordinates: 103.851105557934, 1.3167570723148
- block 52 coordinates: 103.851611305628, 1.31606256409181
- block 53 coordinates: 103.852091950414, 1.31652542675346
- block 54 coordinates: 103.852598943471, 1.31707725396013

We will be removing these rows from the original combined df and appending them to a new df: df_kent. Using df_kent, we will rerun the nearest amenities distance function to get the distance for nearest mrt station, primary, secondary school and mall.

In [71]:
df_kent = df[df['street_name'] == 'KENT RD']

In [72]:
# empty the wrong geo coordinates
df_kent['longitude'] = ''
df_kent['latitude'] = ''

C:\Users\caspe\anaconda3\envs\capstone\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\caspe\anaconda3\envs\capstone\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [73]:
df_kent.columns

Index(['month', 'town', 'flat_type', 'block', 'street_name', 'storey_range',
       'floor_area_sqm', 'flat_model', 'lease_commence_date', 'resale_price',
       'longitude', 'latitude', 'year_transacted', 'age',
       'resale_price_inflate', 'psf', 'floor_area_sqf',
       'nearest_mrt_station_dist', 'nearest_mrt_station',
       'nearest_primary_school_dist', 'nearest_primary_school',
       'nearest_secondary_school_dist', 'nearest_secondary_school',
       'nearest_mall_dist', 'nearest_mall', 'dist_cbd', 'avg_storey'],
      dtype='object')

In [74]:
# dropping all these distance columns to recalculate them again
df_kent.drop(columns=['nearest_mrt_station_dist', 'nearest_mrt_station',
       'nearest_primary_school_dist', 'nearest_primary_school',
       'nearest_secondary_school_dist', 'nearest_secondary_school',
       'nearest_mall_dist', 'nearest_mall'], inplace=True)

C:\Users\caspe\anaconda3\envs\capstone\lib\site-packages\pandas\core\frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [75]:
# filling the right geo coordinates into the rows
df_kent.loc[df_kent['block'] == '51', 'longitude'] = 103.851105557934
df_kent.loc[df_kent['block'] == '51', 'latitude'] = 1.3167570723148
df_kent.loc[df_kent['block'] == '52', 'longitude'] = 103.851611305628
df_kent.loc[df_kent['block'] == '52', 'latitude'] = 1.31606256409181
df_kent.loc[df_kent['block'] == '53', 'longitude'] = 103.852091950414
df_kent.loc[df_kent['block'] == '53', 'latitude'] = 1.31652542675346
df_kent.loc[df_kent['block'] == '54', 'longitude'] = 103.852598943471
df_kent.loc[df_kent['block'] == '54', 'latitude'] = 1.31707725396013

C:\Users\caspe\anaconda3\envs\capstone\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [76]:
# resetting the index
df_kent.index = np.arange(len(df_kent))

In [35]:
# recalculating all the distances from nearest amenities
#get_distance_km_amenity(df_kent, mrt, 'mrt_station')

![](images/get_dist.jpg)

In [36]:
#get_distance_km_amenity(df_kent, pri_sch, 'primary_school')

![](images/get_dist.jpg)

In [37]:
#get_distance_km_amenity(df_kent, sec_sch, 'secondary_school')

![](images/get_dist.jpg)

In [38]:
#get_distance_km_amenity(df_kent, mall, 'mall')

![](images/get_dist.jpg)

In [81]:
# drop these incorrect rows in main df
df = df[~df['street_name'].str.contains('KENT RD')]

In [82]:
df.columns

Index(['month', 'town', 'flat_type', 'block', 'street_name', 'storey_range',
       'floor_area_sqm', 'flat_model', 'lease_commence_date', 'resale_price',
       'longitude', 'latitude', 'year_transacted', 'age',
       'resale_price_inflate', 'psf', 'floor_area_sqf',
       'nearest_mrt_station_dist', 'nearest_mrt_station',
       'nearest_primary_school_dist', 'nearest_primary_school',
       'nearest_secondary_school_dist', 'nearest_secondary_school',
       'nearest_mall_dist', 'nearest_mall', 'dist_cbd', 'avg_storey'],
      dtype='object')

In [83]:
# reordering the columns to match the main df before concatenating both of them together 
df_kent = df_kent[['month', 'town', 'flat_type', 'block', 'street_name', 'storey_range',
       'floor_area_sqm', 'flat_model', 'lease_commence_date', 'resale_price',
       'longitude', 'latitude', 'year_transacted', 'age',
       'resale_price_inflate', 'psf', 'floor_area_sqf',
       'nearest_mrt_station_dist', 'nearest_mrt_station',
       'nearest_primary_school_dist', 'nearest_primary_school',
       'nearest_secondary_school_dist', 'nearest_secondary_school',
       'nearest_mall_dist', 'nearest_mall', 'dist_cbd', 'avg_storey']]

In [84]:
df_kent

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,...,nearest_mrt_station_dist,nearest_mrt_station,nearest_primary_school_dist,nearest_primary_school,nearest_secondary_school_dist,nearest_secondary_school,nearest_mall_dist,nearest_mall,dist_cbd,avg_storey
0,2011-10-01,KALLANG/WHAMPOA,4 ROOM,52,KENT RD,07 TO 09,93.0,New Generation,1982,475000.0,...,0.506825,FARRER PARK,0.421493,FARRER PARK PRIMARY SCHOOL,1.352936,BENDEMEER SECONDARY SCHOOL,0.706908,SERANGOON PLAZA,6.793800,8.0
1,2012-02-01,KALLANG/WHAMPOA,4 ROOM,52,KENT RD,10 TO 12,93.0,New Generation,1982,510000.0,...,0.506825,FARRER PARK,0.421493,FARRER PARK PRIMARY SCHOOL,1.352936,BENDEMEER SECONDARY SCHOOL,0.706908,SERANGOON PLAZA,6.793800,11.0
2,2012-03-01,KALLANG/WHAMPOA,4 ROOM,53,KENT RD,01 TO 05,91.0,New Generation,1982,487000.0,...,0.536474,FARRER PARK,0.485302,FARRER PARK PRIMARY SCHOOL,1.278736,BENDEMEER SECONDARY SCHOOL,0.723666,SERANGOON PLAZA,3.625895,3.0
3,2012-04-01,KALLANG/WHAMPOA,5 ROOM,54,KENT RD,06 TO 10,127.0,Improved,1982,688000.0,...,0.583486,FARRER PARK,0.562003,FARRER PARK PRIMARY SCHOOL,1.195675,BENDEMEER SECONDARY SCHOOL,0.755497,SERANGOON PLAZA,6.759733,8.0
4,2012-08-01,KALLANG/WHAMPOA,5 ROOM,54,KENT RD,01 TO 03,118.0,Improved,1982,580000.0,...,0.583486,FARRER PARK,0.562003,FARRER PARK PRIMARY SCHOOL,1.195675,BENDEMEER SECONDARY SCHOOL,0.755497,SERANGOON PLAZA,6.759733,2.0
5,2013-03-01,KALLANG/WHAMPOA,4 ROOM,53,KENT RD,01 TO 03,91.0,New Generation,1982,530000.0,...,0.536474,FARRER PARK,0.485302,FARRER PARK PRIMARY SCHOOL,1.278736,BENDEMEER SECONDARY SCHOOL,0.723666,SERANGOON PLAZA,3.625895,2.0
6,2013-04-01,KALLANG/WHAMPOA,4 ROOM,52,KENT RD,07 TO 09,91.0,New Generation,1982,530000.0,...,0.506825,FARRER PARK,0.421493,FARRER PARK PRIMARY SCHOOL,1.352936,BENDEMEER SECONDARY SCHOOL,0.706908,SERANGOON PLAZA,6.793800,8.0
7,2013-04-01,KALLANG/WHAMPOA,4 ROOM,52,KENT RD,07 TO 09,91.0,New Generation,1982,538000.0,...,0.506825,FARRER PARK,0.421493,FARRER PARK PRIMARY SCHOOL,1.352936,BENDEMEER SECONDARY SCHOOL,0.706908,SERANGOON PLAZA,6.793800,8.0
8,2013-04-01,KALLANG/WHAMPOA,5 ROOM,54,KENT RD,13 TO 15,118.0,Improved,1982,650000.0,...,0.583486,FARRER PARK,0.562003,FARRER PARK PRIMARY SCHOOL,1.195675,BENDEMEER SECONDARY SCHOOL,0.755497,SERANGOON PLAZA,6.759733,14.0
9,2013-06-01,KALLANG/WHAMPOA,4 ROOM,53,KENT RD,04 TO 06,91.0,New Generation,1982,521000.0,...,0.536474,FARRER PARK,0.485302,FARRER PARK PRIMARY SCHOOL,1.278736,BENDEMEER SECONDARY SCHOOL,0.723666,SERANGOON PLAZA,3.625895,5.0


In [85]:
df = pd.concat([df,df_kent], axis=0)
df

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,...,nearest_mrt_station_dist,nearest_mrt_station,nearest_primary_school_dist,nearest_primary_school,nearest_secondary_school_dist,nearest_secondary_school,nearest_mall_dist,nearest_mall,dist_cbd,avg_storey
0,2011-10-01,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,01 TO 03,60.0,Improved,1986,283000.0,...,0.420801,MAYFLOWER,0.232987,MAYFLOWER PRIMARY SCHOOL JAN 2019 - DEC 2021,0.595743,YIO CHU KANG SECONDARY SCHOOL,0.990781,BROADWAY PLAZA,10.256373,2.0
1,2011-10-01,ANG MO KIO,3 ROOM,331,ANG MO KIO AVE 1,10 TO 12,68.0,New Generation,1981,343000.0,...,0.880294,ANG MO KIO,0.387262,TECK GHEE PRIMARY SCHOOL,0.475333,DEYI SECONDARY SCHOOL,0.848568,AMK HUB,8.696086,11.0
2,2011-10-01,ANG MO KIO,3 ROOM,441,ANG MO KIO AVE 10,04 TO 06,67.0,New Generation,1979,333000.0,...,0.670305,ANG MO KIO,0.360764,TECK GHEE PRIMARY SCHOOL,0.18052,DEYI SECONDARY SCHOOL,0.733697,AMK HUB,9.138814,5.0
3,2011-10-01,ANG MO KIO,3 ROOM,442,ANG MO KIO AVE 10,10 TO 12,82.0,New Generation,1979,399000.0,...,0.790373,ANG MO KIO,0.475436,TECK GHEE PRIMARY SCHOOL,0.301163,DEYI SECONDARY SCHOOL,0.858353,AMK HUB,9.099361,11.0
4,2011-10-01,ANG MO KIO,3 ROOM,435,ANG MO KIO AVE 10,04 TO 06,82.0,New Generation,1979,383000.0,...,0.574072,ANG MO KIO,0.43194,TECK GHEE PRIMARY SCHOOL,0.265444,DEYI SECONDARY SCHOOL,0.664854,AMK HUB,9.322790,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45,2021-01-01,KALLANG/WHAMPOA,4 ROOM,52,KENT RD,01 TO 03,91.0,New Generation,1982,435000.0,...,0.506825,FARRER PARK,0.421493,FARRER PARK PRIMARY SCHOOL,1.352936,BENDEMEER SECONDARY SCHOOL,0.706908,SERANGOON PLAZA,6.793805,2.0
46,2021-01-01,KALLANG/WHAMPOA,4 ROOM,51,KENT RD,07 TO 09,91.0,New Generation,1982,480000.0,...,0.600654,FARRER PARK,0.489589,FARRER PARK PRIMARY SCHOOL,1.344834,BENDEMEER SECONDARY SCHOOL,0.802395,SERANGOON PLAZA,3.651203,8.0
47,2021-02-01,KALLANG/WHAMPOA,5 ROOM,54,KENT RD,01 TO 03,120.0,Improved,1982,640888.0,...,0.583486,FARRER PARK,0.562003,FARRER PARK PRIMARY SCHOOL,1.195675,BENDEMEER SECONDARY SCHOOL,0.755497,SERANGOON PLAZA,6.759735,2.0
48,2021-03-01,KALLANG/WHAMPOA,4 ROOM,52,KENT RD,04 TO 06,91.0,New Generation,1982,538000.0,...,0.506825,FARRER PARK,0.421493,FARRER PARK PRIMARY SCHOOL,1.352936,BENDEMEER SECONDARY SCHOOL,0.706908,SERANGOON PLAZA,6.793805,5.0


In [86]:
df['month'] = pd.to_datetime(df['month'])
df.set_index('month', inplace=True)

In [88]:
df.sort_index(inplace=True)

In [102]:
# save combined df
df.to_csv('../datasets/final/hdb_combine (3 Nov).csv')